# --- Day 5: Alchemical Reduction ---

You've managed to sneak in to the prototype suit manufacturing lab. The Elves are making decent progress, but are still struggling with the suit's size reduction capabilities.

While the very latest in 1518 alchemical technology might have solved their problem eventually, you can do better. You scan the chemical composition of the suit's material and discover that it is formed by extremely long polymers (one of which is available as your puzzle input).

The polymer is formed by smaller units which, when triggered, react with each other such that two adjacent units of the same type and opposite polarity are destroyed. Units' types are represented by letters; units' polarity is represented by capitalization. For instance, `r` and `R` are units with the same type but opposite polarity, whereas `r` and `s` are entirely different types and do not react.

For example:

- In `aA`, `a` and `A` react, leaving nothing behind.
- In `abBA`, bB destroys itself, leaving `aA`. As above, this then destroys itself, leaving nothing.
- In `abAB`, no two adjacent units are of the same type, and so nothing happens.
- In `aabAAB`, even though `aa` and `AA` are of the same type, their polarities match, and so nothing happens.

Now, consider a larger example, `dabAcCaCBAcCcaDA`:

```
dabAcCaCBAcCcaDA  The first 'cC' is removed.
dabAaCBAcCcaDA    This creates 'Aa', which is removed.
dabCBAcCcaDA      Either 'cC' or 'Cc' are removed (the result is the same).
dabCBAcaDA        No further actions can be taken.
```

After all possible reactions, the resulting polymer contains 10 units.

How many units remain after fully reacting the polymer you scanned?

In [22]:
import itertools as it
import re
import string

In [20]:
def input_():
    with open('./inputs/day05/input.txt') as fp:
        return fp.read().strip()

In [4]:
def sliding_window(input_str):
    """
    Iterate over an input string pairwise,
    yielding tuples of ((index, character), (index, characater)).
    
    """
    first = enumerate(it.islice(input_str, 0, len(input_str) - 1), start=0)
    second = enumerate(it.islice(input_str, 1, None), start=1)
    
    yield from zip(first, second)

In [5]:
list(sliding_window('abcd'))

[((0, 'a'), (1, 'b')), ((1, 'b'), (2, 'c')), ((2, 'c'), (3, 'd'))]

In [7]:
def destruct(c1, c2):
    return c1.lower() == c2.lower() and c1 != c2

In [8]:
destruct('a', 'A'), destruct('a', 'a'), destruct('a', 'b'), destruct('a', 'C')

(True, False, False, False)

In [16]:
def condense(input_str):
    input_iter = sliding_window(input_str)
    to_remove = set()
    
    while True:
        try:
            (idx1, c1), (idx2, c2) = next(input_iter)
        except StopIteration:
            break
            
        if destruct(c1, c2):
            to_remove.update({idx1, idx2})
            
            # skip the next iteration to get past c2
            try:
                next(input_iter)
            except StopIteration:
                break
    
    result = ''.join(c for idx, c in enumerate(input_str) if idx not in to_remove)
    
    if not to_remove:
        return result
    else:
        return condense(result)

In [17]:
condense('abcd')

'abcd'

In [19]:
print(condense('dabAcCaCBAcCcaDA'), 'expected', 'dabCBAcaDA')

dabCBAcaDA expected dabCBAcaDA


In [21]:
print(len(condense(input_())))

9288


# --- Part Two ---

Time to improve the polymer.

One of the unit types is causing problems; it's preventing the polymer from collapsing as much as it should. Your goal is to figure out which unit type is causing the most problems, remove all instances of it (regardless of polarity), fully react the remaining polymer, and measure its length.

For example, again using the polymer `dabAcCaCBAcCcaDA` from above:

- Removing all `A/a` units produces `dbcCCBcCcD`. Fully reacting this polymer produces `dbCBcD`, which has length 6.
- Removing all `B/b` units produces `daAcCaCAcCcaDA`. Fully reacting this polymer produces `daCAcaDA`, which has length 8.
- Removing all `C/c` units produces `dabAaBAaDA`. Fully reacting this polymer produces `daDA`, which has length 4.
- Removing all `D/d` units produces `abAcCaCBAcCcaA`. Fully reacting this polymer produces `abCBAc`, which has length 6.

In this example, removing all `C/c` units was best, producing the answer 4.

What is the length of the shortest polymer you can produce by removing all units of exactly one type and fully reacting the result?

In [23]:
def remove_char(input_str, char):
    return re.sub(char, '', input_str, flags=re.IGNORECASE)

In [25]:
min(len(condense(remove_char('dabAcCaCBAcCcaDA', char))) for char in 'abcd')

4

In [27]:
input_str = input_()
print(min(len(condense(remove_char(input_str, char))) for char in string.ascii_lowercase))

5844
